# 

In [1]:
!pip install arsenic

     |████████████████████████████████| 49 kB 12.5 MB/s 
     |████████████████████████████████| 1.3 MB 43.6 MB/s 
     |████████████████████████████████| 294 kB 73.6 MB/s 
     |████████████████████████████████| 142 kB 78.3 MB/s 


In [8]:
%%writefile async_scrape.py
import os
import asyncio
from arsenic import get_session, keys, browsers, services
import pandas as pd
from requests_html import HTML 
import itertools
import re
import time
import pathlib
import pickle


async def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w-]+)$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']

async def get_links(body_content):
    html_r = HTML(html=body_content)
    fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]

    datas = []
    for path in fabric_links:
        id_, slug_ = extract_id_slug(path)
        data = {
            'id': id_,
            'slug': slug_,
            'path': path,
            'scraped': 0
        }
        datas.append(data)
    return datas

async def scraper(url):
    service = services.Chromedriver()
    browser = browsers.Chrome(chromeOption={
        'args': ['--headless', '--disable-gpu']
    })

    async with get_session(service, browser) as session:
        await session.get(url)
        body = await session.get_page_source()
        return body

async def store_links_as_df_pickle(datas=[], name='links.pkl'):
    df = pd.DataFrame(datas)
    df.set_index('id', drop=True, inplace=True)
    df.to_pickle(name)
    return df

async def run(url):
    body_content = await scraper(url)
    links = await get_links(body_content)
    return df

if __name__ == "__main__":
    url = 'https://www.spoonflower.com/en/shop?on=fabric'
    # loop = asyncio.get_event_loop()
    # result = loop.run_until_complete(scraper(url))
    results = asyncio.run(scraper(url))
    print(results)

Overwriting async_scrape.py


In [9]:
!python async_scrape.py

Traceback (most recent call last):
  File "async_scrape.py", line 36, in <module>
    result = asyncio.run(scraper(url))
  File "/usr/local/lib/python3.7/asyncio/runners.py", line 43, in run
    return loop.run_until_complete(main)
  File "/usr/local/lib/python3.7/asyncio/base_events.py", line 587, in run_until_complete
    return future.result()
  File "async_scrape.py", line 26, in scraper
    async with get_session(service, browser) as session:
  File "/root/venv/lib/python3.7/site-packages/arsenic/__init__.py", line 16, in __aenter__
    self.session = await start_session(self.service, self.browser, self.bind)
  File "/root/venv/lib/python3.7/site-packages/arsenic/__init__.py", line 28, in start_session
    driver = await service.start()
  File "/root/venv/lib/python3.7/site-packages/arsenic/services.py", line 108, in start
    [self.binary, f"--port={port}"], f"http://localhost:{port}", self.log_file
  File "/root/venv/lib/python3.7/site-packages/arsenic/services.py", line 34, in 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=455dbd66-b743-41fb-bb10-e0d89ff02cc3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>